In [172]:
import numpy as np
import pandas as pd
import datetime

In [285]:
raw_data = pd.read_csv('training.csv')
pd.set_option('display.max_rows', None)

In [286]:
def clean_transform(raw_data):
    #Drop duplicates
    raw_data = raw_data.drop_duplicates()
    
    #Change data types, column names, percentages to decimals
    rd = raw_data.astype({'season': str, 'starter': str, 'did_not_play': str, 'is_inactive': str})
    rd['game_date'] = pd.to_datetime(rd['game_date'], infer_datetime_format = True)
    rd = rd.rename(columns = {'Team_efg_pct': 'Team_efg_rate', 'Team_tov_pct' : 'Team_tov_rate', 
                              'Team_orb_pct': 'Team_orb_rate', 'Opponent_efg_pct': 'Opponent_efg_rate', 
                              'Opponent_tov_pct': 'Opponent_tov_rate', 'Opponent_orb_pct': 'Opponent_orb_rate',
                             'ts_pct': 'ts_rate', 'efg_pct': 'efg_rate', 'ts_pct': 'ts_rate', 'orb_pct': 'orb_rate', 
                             'drb_pct': 'drb_rate', 'trb_pct': 'trb_rate', 'ast_pct': 'ast_rate', 
                              'stl_pct': 'stl_rate','blk_pct': 'blk_rate', 'tov_pct': 'tov_rate', 
                              'usg_pct': 'usg_rate', 'ft_pct': 'ft_rate', 'fg_pct': 'fg_rate', 'fg3_pct': 'fg3_rate'})
    rd[['Team_tov_rate', 'Team_orb_rate', 'Opponent_orb_rate', 'Opponent_tov_rate', 'orb_rate', 
        'drb_rate', 'trb_rate', 'ast_rate', 'stl_rate', 'blk_rate', 'tov_rate', 'usg_rate']] = rd[['Team_tov_rate', 'Team_orb_rate', 'Opponent_orb_rate', 'Opponent_tov_rate', 'orb_rate', 
        'drb_rate', 'trb_rate', 'ast_rate', 'stl_rate', 'blk_rate', 'tov_rate', 'usg_rate']]/100
    raw_data = rd
    
    #Create inactives table
    global inactives
    inactives = raw_data[['game_id', 'Inactives']]
    
    #Drop rows for games that went to overtime
    raw_data = raw_data[raw_data['OT'] == 0]
    
    #Remove unnecessary columns
    global clean_data
    clean_data = raw_data.drop(['OT', 'player', 'mp', 'double_double', 'triple_double', 'active_position_minutes', 
                   'PG%', 'SG%', 'SF%', 'C%', 'Inactives', 'pf', 'is_inactive', 'PF%',
                    'last_60_minutes_per_game_starting', 'last_60_minutes_per_game_bench',
                    'DKP', 'FDP', 'SDP', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pts', 'ts'], axis = 1, 
                               inplace = False)

In [287]:
clean_transform(raw_data)

In [231]:
def describe(dataframe):
    print('Shape: ', dataframe.shape)
    print(dataframe.columns)
    print(dataframe.dtypes)

In [182]:
def transpose(dataframe):
    transpose = dataframe.iloc[0:9].T
    return transpose.head(len(dataframe))

In [252]:
def pred_player_stats(clean_data):
    sorted = clean_data.sort_values(by = ['player_id', 'game_date'])
    return sorted.head(100)

In [271]:
clean_data[clean_data['player_id'] == 'achiupr01']

,game_id,game_date,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_rate,Team_tov_rate,Team_orb_rate,Team_ft_rate,...,usg_rate,off_rtg,def_rtg,bpm,season,minutes,DKP_per_minute,FDP_per_minute,SDP_per_minute,pf_per_minute
11910,202012230ORL,2020-12-23,A,MIA,107,104.9,0.548,0.194,0.189,0.193,...,0.179,142,112,-1.6,2021,13.650000,0.860806,0.849817,0.860806,0.146520
11911,202012230ORL,2020-12-23,A,MIA,107,104.9,0.548,0.194,0.189,0.193,...,0.179,142,112,-1.6,2021,13.650000,0.860806,0.849817,0.860806,0.146520
11934,202012250MIA,2020-12-25,H,MIA,111,96.8,0.613,0.166,0.118,0.253,...,0.214,128,105,-1.8,2021,19.466667,0.988870,1.006849,1.014555,0.308219
11935,202012250MIA,2020-12-25,H,MIA,111,96.8,0.613,0.166,0.118,0.253,...,0.214,128,105,-1.8,2021,19.466667,0.988870,1.006849,1.014555,0.308219
11958,202101010DAL,2021-01-01,A,MIA,83,97.5,0.417,0.136,0.100,0.231,...,0.297,136,98,14.8,2021,11.983333,1.543811,1.477051,1.502086,0.000000
11959,202101010DAL,2021-01-01,A,MIA,83,97.5,0.417,0.136,0.100,0.231,...,0.297,136,98,14.8,2021,11.983333,1.543811,1.477051,1.502086,0.000000
11982,202101040MIA,2021-01-04,H,MIA,118,98.1,0.667,0.136,0.111,0.123,...,0.195,89,87,-9.9,2021,19.600000,0.829082,0.836735,0.854592,0.153061
11983,202101040MIA,2021-01-04,H,MIA,118,98.1,0.667,0.136,0.111,0.123,...,0.195,89,87,-9.9,2021,19.600000,0.829082,0.836735,0.854592,0.153061
12003,202101060MIA,2021-01-06,H,MIA,105,92.3,0.512,0.080,0.140,0.272,...,0.231,123,116,4.4,2021,14.233333,1.282201,1.299766,1.317330,0.070258
12004,202101060MIA,2021-01-06,H,MIA,105,92.3,0.512,0.080,0.140,0.272,...,0.231,123,116,4.4,2021,14.233333,1.282201,1.299766,1.317330,0.070258


In [247]:
transpose(clean_data)

,0,1,2,3,4,5,6,7,8
game_id,202012220BRK,202012220BRK,202012220BRK,202012220BRK,202012220BRK,202012220BRK,202012220BRK,202012220BRK,202012220BRK
game_date,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00,2020-12-22 00:00:00
H_A,A,A,A,A,A,A,A,A,A
Team_Abbrev,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW,GSW
Team_Score,99,99,99,99,99,99,99,99,99
Team_pace,110.5,110.5,110.5,110.5,110.5,110.5,110.5,110.5,110.5
Team_efg_rate,0.424,0.424,0.424,0.424,0.424,0.424,0.424,0.424,0.424
Team_tov_rate,0.142,0.142,0.142,0.142,0.142,0.142,0.142,0.142,0.142
Team_orb_rate,0.228,0.228,0.228,0.228,0.228,0.228,0.228,0.228,0.228
Team_ft_rate,0.152,0.152,0.152,0.152,0.152,0.152,0.152,0.152,0.152
